In [2]:
#!/usr/bin/env python
import snowflake.connector

import time
import calendar

import hashlib as hashlib
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import sklearn
import sklearn.metrics as sm

from sklearn import datasets
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
from mpl_toolkits.mplot3d import Axes3D

from snowflake.connector.pandas_tools import write_pandas
from snowflake.connector.pandas_tools import pd_writer
from hashlib import md5

from datetime import date, datetime, timedelta
from dateutil.parser import parse as dateParse

In [3]:
#---------------------------------------
# For installing dependencies
#---------------------------------------
import sys
!{sys.executable} -m pip install "snowflake-connector-python[pandas]"
!{sys.executable} -m pip install "matplotlib"
!{sys.executable} -m pip install "scikit-learn"

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
#---------------------------------------
# CONNECT TO SNOWFLAKE
#---------------------------------------

con = snowflake.connector.connect(
    user='codestudio_samplefarm_evening',
    password='ikRsb9D00UNr',
    account='FT76388.ap-southeast-2',
    #warehouse='Main',
    database='AW_INTERNAL_STAGING',
    schema='AW_AURORA_AGRIWEBBDATA'
    )

In [7]:
con

In [9]:
#---------------------------------------
# Test reaching the dataset
#---------------------------------------
con_cursor = con.cursor()
try:
    con_cursor.execute("SELECT UUID, FARMID, TENANTID, TITLE, PARENTUUID, COLOR, ORIGIN, DATEOFBIRTH, \
                       PURCHASEPRICE, LONGITUDE, LATITUDE, SUBTYPE, BREEDS, ASSETTYPE, WEIGHT,\
                        PASTUREID, SIZE \
                        \
                        FROM AW_INTERNAL_STAGING.STAGING_AURORA_AGRIWEBBDATA.MOBLIVESTOCKGROUP")
    
    pd.set_option("max_rows", 1000)
    
    df = con_cursor.fetch_pandas_all()
    
    print(df)
    
finally:
    con_cursor.close()

                                        UUID  \
0       9a258208-94ba-47ac-8f20-7db8aa1821d3   
1       3fd3ef90-dc05-4d97-b9a1-aa2b65f7d473   
2       09d26358-c3e3-4ad2-8a98-91be068397ba   
3       3a96d7a2-60c2-45f7-88b1-54018fab8d74   
4       9c0826ec-3695-4437-a30e-0dccfe699ac2   
...                                      ...   
230557  fffe5b84-fbe7-4a61-8a3e-1efb1236a75b   
230558  fffe6ffb-2053-482a-880f-542bdb822012   
230559  ffff7eb3-9edf-44eb-8da9-bc12f684f89e   
230560                           healthcheck   
230561  58ac8d9d-70f7-4166-8a78-d93345cc3e60   

                                      FARMID  \
0       be63421f-ee14-424e-8f02-b9e5a71a3cd2   
1       3a937427-3315-49c3-9fa2-1a53016287bf   
2       3821ef41-a38e-418c-80a0-2d9ecfcb034f   
3       5b8ccc2e-8b19-4f14-ac5b-3aec9f72c8fd   
4       52934454-d6cf-460b-adc0-05ed5821e6b6   
...                                      ...   
230557  2698b8d0-83c8-4c54-8159-fbc92b1b1270   
230558  a30722ea-e37e-4e42-8584-1bf5b44

In [5]:
#---------------------------------------
# Test clustering algorithm
#---------------------------------------
clustering = KMeans(n_clusters=3, random_state=10)

con_cursor = con.cursor()
try:
    con_cursor.execute("SELECT COLOR, ORIGIN, DATEOFBIRTH, PURCHASEPRICE, LONGITUDE, LATITUDE, SUBTYPE, BREEDS, WEIGHT, SIZE \
                        FROM AW_INTERNAL_STAGING.STAGING_AURORA_AGRIWEBBDATA.MOBLIVESTOCKGROUP")
    
    pd.set_option("max_rows", 1000)
    
    df = con_cursor.fetch_pandas_all()
    
#     # Preprocessing
#     enc = OneHotEncoder()
#     enc.fit(df)
    
#     enc_df = pd.DataFrame(enc.fit_transform(df[['ORIGIN']]).toarray())
#     # merge with main df bridge_df on key values
#     df = df.join(enc_df)
#     print(df)
    
    origin_df = pd.get_dummies(df, columns=["ORIGIN", "SUBTYPE", "BREEDS"], 
                               prefix=["Origin_", "Subtype_", "Breeds_"], drop_first=True)
    
    df.merge(origin_df, how='left')
    
    print(df)
    
    #clustering.fit(df)
    
finally:
    con_cursor.close()

Error: Canceled future for execute_request message before replies were done